## Problem 3: How long distance individuals have travelled? (8 points)

In this problem the aim is to calculate the "distance" in meters that the individuals have travelled according the social media posts (Euclidean distances between points). In this problem, we will need the `userid` -column an the points created in the previous problem. You will need the shapefile `Kruger_posts.shp` generated in Problem 2 as input file.

Our goal is to answer these questions based on the input data:
- What was the shortest distance travelled in meters?
- What was the mean distance travelled in meters?
- What was the maximum distance travelled in meters?

**In your code, you should first:**
 - Import required modules.
 - Read in the shapefile as a geodataframe called `data`
 - Reproject the data from WGS84 projection into `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into metric system.
 
*Store the result in a variable called `data`*!

In [47]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon, LineString
from pyproj import CRS

In [31]:
import os

# Join folder path and filename 
fp = "Kruger_posts.shp"

# Read file using gpd.read_file()
data = gpd.read_file(fp)

In [32]:
# Reproject the data
data = data.to_crs(epsg=32735)

- Check the crs of the input data. If this information is missing, set it as epsg:4326 (WGS84).
- Reproject the data from WGS84 to `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into metric system. (don't create a new variable, update the existing variable `data`!)"

In [33]:
data.crs

<Projected CRS: EPSG:32735>
Name: WGS 84 / UTM zone 35S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 24°E and 30°E, southern hemisphere between 80°S and equator, onshore and offshore. Botswana. Burundi. Democratic Republic of the Congo (Zaire). Rwanda. South Africa. Tanzania. Uganda. Zambia. Zimbabwe.
- bounds: (24.0, -80.0, 30.0, 0.0)
Coordinate Operation:
- name: UTM zone 35S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [34]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the data
print(data.head())

         lat        lon         timestamp    userid  \
0 -24.980792  31.484633  2015-07-07 03:02  66487960   
1 -25.499225  31.508906  2015-07-07 03:18  65281761   
2 -24.342578  30.930866  2015-03-07 03:38  90916112   
3 -24.854614  31.519718  2015-10-07 05:04  37959089   
4 -24.921069  31.520836  2015-10-07 05:19  27793716   

                         geometry  
0  POINT (952912.890 7229683.258)  
1  POINT (953433.223 7172080.632)  
2  POINT (898955.144 7302197.408)  
3  POINT (956927.218 7243564.942)  
4  POINT (956794.955 7236187.926)  


In [35]:
data.tail()

,lat,lon,timestamp,userid,geometry
81374,-24.799541,31.354469,2015-09-05 02:23,90744213,POINT (940395.058 7250220.045)
81375,-25.467992,30.956033,2015-02-05 02:40,71109799,POINT (897877.380 7177317.787)
81376,-25.332223,30.997409,2015-08-05 02:40,54796261,POINT (902494.313 7192250.813)
81377,-25.508851,31.005536,2015-08-05 02:43,78762204,POINT (902724.670 7172637.035)
81378,-25.498840,31.007430,2015-04-05 02:44,32190273,POINT (902948.779 7173741.587)


In [36]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check that the crs is correct after re-projecting (should be epsg:32735)
print(data.crs)

epsg:32735


 - Group the data by userid

In [37]:
grouped = data.groupby('userid')

In [38]:
print(grouped)

In [39]:
len(grouped)

14990

In [40]:
data['userid'].nunique()

14990

In [41]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the number of groups:
assert len(grouped.groups) == data["userid"].nunique(), "Number of groups should match number of unique users!"

In [50]:
grouped.head()

,lat,lon,timestamp,userid,geometry
0,-24.980792,31.484633,2015-07-07 03:02,66487960,POINT (952912.890 7229683.258)
1,-25.499225,31.508906,2015-07-07 03:18,65281761,POINT (953433.223 7172080.632)
2,-24.342578,30.930866,2015-03-07 03:38,90916112,POINT (898955.144 7302197.408)
3,-24.854614,31.519718,2015-10-07 05:04,37959089,POINT (956927.218 7243564.942)
4,-24.921069,31.520836,2015-10-07 05:19,27793716,POINT (956794.955 7236187.926)
...,...,...,...,...,...
81361,-25.431166,31.952533,2015-11-04 20:38,120615,POINT (998394.873 7178043.198)
81368,-25.385833,31.828889,2015-04-05 01:10,5970719,POINT (986115.441 7183532.368)
81372,-25.080739,31.918978,2015-08-05 01:52,46410906,POINT (996441.120 7217063.910)
81374,-24.799541,31.354469,2015-09-05 02:23,90744213,POINT (940395.058 7250220.045)


**Create LineString objects for each user connecting the points from oldest to latest:**

*Suggested steps:*
- Create an empty DataFrame called `movements`. 
- Create an empty column "geometry"
- Use a for-loop where you iterate over the grouped object. For each user's data: 
    - [sort](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) the rows by timestamp 
    - create a LineString object based on the user's points
    - Add the LineString to the geometry column of the `movements` dataframe. You can also add the `userid` in a separate column (or use the userid as index).
- Convert `movements` into a `GeoDataFrame` (you can replace the DataFrame created in the previous steps with the GeoDataFrame). Remember to set the `geometry` column.
- Set the CRS of the ``movements`` GeoDataFrame as ``EPSG:32735`` 

In [54]:
# Create an empty GeoDataFrame
movements = pd.DataFrame()

movements['geometry'] = None

#iterate over grouped object
for key, values in grouped:

    #sort rows by timestamp
    grouped = grouped.sort_values(['timestamp'], ascending=True)
    
#Iterate over dataframe rows:
for idx, row in data.iterrows():

    # create a point based on x and y column values on this row:
    point = Point(row['lon'], row['lat'])

    # Add the point object to the geometry column on this row:
    data.at[idx, 'geometry'] = point

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_values'

In [ ]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the result
print(type(movements))
print(movements.crs)
print(movements["geometry"].head())

**Finally:**
- Check once more the crs definition of your dataframe (should be epsg:32735, define the correct crs if this information is missing)
- Calculate the lenghts of the lines into a new column called ``distance`` in ``movements`` GeoDataFrame.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

In [ ]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the output
movements.head()

You should now be able to print answers to the following questions: 

 - What was the shortest distance travelled in meters?
 - What was the mean distance travelled in meters?
 - What was the maximum distance travelled in meters?

- Finally, save the movements of into a Shapefile called ``some_movements.shp``

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

In [ ]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

import os

#Check if output file exists
assert os.path.isfile(fp), "Output file does not exits."

That's all for this week!